# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [46]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# метод get
habr = requests.get('https://habr.com/ru/all/')

# res
habr.status_code

200

In [5]:
habr.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Все публикации подряд / Хабр</title>\n\n\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:type" content="website"/>\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:site_name" content="Хабр" />\n<link rel="image_src" href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image" content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image:width" content="1200" />\n<meta property="og:image:height" content="628" />\n<meta property="og:title" content="Все публикации подряд / Хабр"/>\n<meta property="og:description" content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом."/>\n\n\n\n<meta name=\'yandex-verification\

In [8]:
KEYWORDS = ['python', 'парсинг']

In [10]:
soup = BeautifulSoup(habr.text, 'html.parser')

In [43]:
# добираемся до блоков с новостями
posts = soup.find_all('h2', class_='post__title')
posts

[<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/skillfactory/blog/558672/">3 способа визуального извлечения данных с помощью JavaScript</a>
 </h2>, <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/559228/">Как я хотел поработать нативным Android разработчиком, но устроился Flutter разрабом</a>
 </h2>, <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/englishdom/blog/559100/">Good vs. Well, или Гайд о прилагательных и наречиях в английском языке</a>
 </h2>, <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/selectel/blog/555948/">Процессорам Alder Lake от Intel понадобятся новые системы охлаждения и материнские платы</a>
 </h2>, <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/559220/">Deepfake. Реализация технологии замены лиц в видео. Часть 1</a>
 </h2>, <h2 class="post__title">
 <a class="post__titl

In [44]:
articles_links = list(map(lambda x: x.find('a').get('href'), posts))
articles_links

['https://habr.com/ru/company/skillfactory/blog/558672/',
 'https://habr.com/ru/post/559228/',
 'https://habr.com/ru/company/englishdom/blog/559100/',
 'https://habr.com/ru/company/selectel/blog/555948/',
 'https://habr.com/ru/post/559220/',
 'https://habr.com/ru/company/deutschetelekomitsolutions/blog/559214/',
 'https://habr.com/ru/post/559212/',
 'https://habr.com/ru/company/wrike/blog/558070/',
 'https://habr.com/ru/company/otus/blog/559210/',
 'https://habr.com/ru/company/ruvds/blog/559148/',
 'https://habr.com/ru/company/otus/blog/559170/',
 'https://habr.com/ru/company/otus/blog/559176/',
 'https://habr.com/ru/company/microsoft/blog/559174/',
 'https://habr.com/ru/post/559146/',
 'https://habr.com/ru/post/559050/',
 'https://habr.com/ru/company/selectel/blog/555824/',
 'https://habr.com/ru/post/559160/',
 'https://habr.com/ru/company/ozontech/blog/557178/',
 'https://habr.com/ru/company/celsus/blog/559150/',
 'https://habr.com/ru/company/macloud/blog/559108/']

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [103]:
kom_news = pd.DataFrame()
for link in articles_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    title = soup.find('title').text  
    text = soup.find(class_='post__body post__body_full').text
    date = pd.to_datetime(soup.find(class_='post__time').get('data-time_published'))
    row = {'date': date, 'title': title,'link': link, 'text': text}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news.reset_index()

,index,date,title,link,text
0,0,2021-05-25 15:18:00+00:00,3 способа визуального извлечения данных с помо...,https://habr.com/ru/company/skillfactory/blog/...,\nК старту курса о Frontend-разработке мы реши...
1,0,2021-05-25 14:56:00+00:00,Как я хотел поработать нативным Android разраб...,https://habr.com/ru/post/559228/,\nНебольшое вступлениеПосле праздничных канику...
2,0,2021-05-25 14:51:00+00:00,"Good vs. Well, или Гайд о прилагательных и нар...",https://habr.com/ru/company/englishdom/blog/55...,\nСегодня мы поговорим о достаточно скользкой ...
3,0,2021-05-25 14:38:00+00:00,Процессорам Alder Lake от Intel понадобятся но...,https://habr.com/ru/company/selectel/blog/555948/,\n\r\nСовсем недавно в нашем блоге публиковала...
4,0,2021-05-25 14:35:00+00:00,Deepfake. Реализация технологии замены лиц в в...,https://habr.com/ru/post/559220/,\nВсем привет! Я работаю с генеративными модел...
5,0,2021-05-25 14:16:00+00:00,«Главное – мотивированность и настойчивость»: ...,https://habr.com/ru/company/deutschetelekomits...,\nКак выглядит типичный процесс отбора на стаж...
6,0,2021-05-25 14:13:00+00:00,Как баг с потерянными днями рождения привёл на...,https://habr.com/ru/post/559212/,\nНа корпоративном портале одного нашего заказ...
7,0,2021-05-25 14:12:00+00:00,Как превратиться в суперзвезду Zoom-звонков за...,https://habr.com/ru/company/wrike/blog/558070/,\n«Ну наконец-то!» — услышал я от нескольких ч...
8,0,2021-05-25 14:11:00+00:00,Создание функции губки из MD5 / Блог компании ...,https://habr.com/ru/company/otus/blog/559210/,"\nПривет, Хабр. В преддверии старта курса ""Pyt..."
9,0,2021-05-25 14:01:00+00:00,"Рояль над котиком, день первый / Блог компании...",https://habr.com/ru/company/ruvds/blog/559148/,"\nЕсли вы ещё не слышали, вчера одна эксцентри..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [105]:
EMAIL = ['admin@gmail.com']

In [106]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [118]:
params = {
'POST': '/v1/web/query/site-breaches/unauthorized-data HTTP/1.1',
'Host': 'identityprotection.avast.com',
'Connection': 'keep-alive',
'Content-Length': '41',
'Accept': 'application/json, text/plain, */*',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Version': '0',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.96 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Content-Type': 'application/json;charset=UTF-8',
'Origin': 'https://www.avast.com',
'Sec-Fetch-Site': 'same-site',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Dest': 'empty',
'Referer': 'https://www.avast.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru,en;q=0.9',
}

In [122]:
res = requests.post(URL, data = params)
res

<Response [400]>

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
